# Compare data mart `_dev` tables to production tables before pushing to production

In [36]:
import pandas as pd
import numpy as np

from google.cloud import bigquery

In [8]:
pd.set_option('display.max_columns', None)

In [3]:
client = bigquery.Client()

In [4]:
prod_county_wide_sql = """
    SELECT *
    FROM `dbcp-dev-350818.data_mart.counties_wide_format`
"""

prod_county_wide_df = client.query_and_wait(prod_county_wide_sql).to_dataframe()

/app/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2575: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/app/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2589: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/app/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2603: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [5]:
dev_county_wide_sql = """
    SELECT *
    FROM `dbcp-dev-350818.data_mart_dev.counties_wide_format`
"""

dev_county_wide_df = client.query_and_wait(dev_county_wide_sql).to_dataframe()

In [6]:
len(prod_county_wide_df), len(dev_county_wide_df)

(3234, 3234)

In [9]:
dev_county_wide_df.head()

,state_id_fips,county_id_fips,state,county,county_total_co2e_tonnes_per_year,fossil_existing_capacity_mw,fossil_existing_co2e_tonnes_per_year,fossil_existing_facility_count,fossil_proposed_capacity_mw,fossil_proposed_co2e_tonnes_per_year,fossil_proposed_facility_count,renewable_and_battery_existing_capacity_mw,renewable_and_battery_existing_co2e_tonnes_per_year,renewable_and_battery_existing_facility_count,renewable_and_battery_proposed_capacity_mw,renewable_and_battery_proposed_facility_count,renewable_and_battery_proposed_avoided_co2e_tonnes_per_year,renewable_and_battery_proposed_capacity_mw_actionable,renewable_and_battery_proposed_facility_count_actionable,renewable_and_battery_proposed_avoided_co2e_actionable,renewable_and_battery_proposed_capacity_mw_nearly_certain,renewable_and_battery_proposed_facility_count_nearly_certain,renewable_and_battery_proposed_avoided_co2e_nearly_certain,infra_total_proposed_co2e_tonnes_per_year,infra_total_proposed_facility_count,infra_total_proposed_nox_tonnes_per_year,infra_total_proposed_pm2_5_tonnes_per_year,battery_storage_existing_capacity_mw,battery_storage_existing_facility_count,battery_storage_existing_co2e_tonnes_per_year,battery_storage_proposed_capacity_mw,battery_storage_proposed_facility_count,coal_existing_capacity_mw,coal_existing_co2e_tonnes_per_year,coal_existing_facility_count,coal_proposed_capacity_mw,coal_proposed_co2e_tonnes_per_year,coal_proposed_facility_count,gas_existing_capacity_mw,gas_existing_co2e_tonnes_per_year,gas_existing_facility_count,gas_proposed_capacity_mw,gas_proposed_co2e_tonnes_per_year,gas_proposed_facility_count,offshore_wind_existing_capacity_mw,offshore_wind_existing_facility_count,offshore_wind_proposed_capacity_mw,offshore_wind_proposed_facility_count,offshore_wind_proposed_avoided_co2e_tonnes_per_year,offshore_wind_capacity_mw_via_ports,offshore_wind_interest_type,offshore_wind_proposed_capacity_mw_actionable,offshore_wind_proposed_facility_count_actionable,offshore_wind_proposed_avoided_co2e_actionable,offshore_wind_proposed_capacity_mw_nearly_certain,offshore_wind_proposed_facility_count_nearly_certain,offshore_wind_proposed_avoided_co2e_nearly_certain,oil_existing_capacity_mw,oil_existing_co2e_tonnes_per_year,oil_existing_facility_count,oil_proposed_capacity_mw,oil_proposed_co2e_tonnes_per_year,oil_proposed_facility_count,onshore_wind_existing_capacity_mw,onshore_wind_existing_facility_count,onshore_wind_proposed_capacity_mw,onshore_wind_proposed_facility_count,onshore_wind_proposed_avoided_co2e_tonnes_per_year,onshore_wind_proposed_capacity_mw_actionable,onshore_wind_proposed_facility_count_actionable,onshore_wind_proposed_avoided_co2e_actionable,onshore_wind_proposed_capacity_mw_nearly_certain,onshore_wind_proposed_facility_count_nearly_certain,onshore_wind_proposed_avoided_co2e_nearly_certain,solar_existing_capacity_mw,solar_existing_co2e_tonnes_per_year,solar_existing_facility_count,solar_proposed_capacity_mw,solar_proposed_facility_count,solar_proposed_avoided_co2e_tonnes_per_year,solar_proposed_capacity_mw_actionable,solar_proposed_facility_count_actionable,solar_proposed_avoided_co2e_actionable,solar_proposed_capacity_mw_nearly_certain,solar_proposed_facility_count_nearly_certain,solar_proposed_avoided_co2e_nearly_certain,infra_gas_proposed_co2e_tonnes_per_year,infra_gas_proposed_facility_count,infra_gas_proposed_nox_tonnes_per_year,infra_gas_proposed_pm2_5_tonnes_per_year,infra_lng_proposed_co2e_tonnes_per_year,infra_lng_proposed_facility_count,infra_lng_proposed_nox_tonnes_per_year,infra_lng_proposed_pm2_5_tonnes_per_year,infra_oil_proposed_co2e_tonnes_per_year,infra_oil_proposed_facility_count,infra_oil_proposed_nox_tonnes_per_year,infra_oil_proposed_pm2_5_tonnes_per_year,infra_petrochemicals_and_plastics_proposed_co2e_tonnes_per_year,infra_petrochemicals_and_plastics_proposed_facility_count,infra_petrochemicals_and_plastics_proposed_nox_tonnes_per_year,infra_petrochemicals_and_plastics_proposed_pm2_5_tonnes_per_yea,infra_ammonia

In [11]:
prod_county_wide_df["dev_prod"] = "prod"
dev_county_wide_df["dev_prod"] = "dev"

In [12]:
cols = [col for col in prod_county_wide_df if col != "dev_prod"]

In [14]:
diff = pd.concat([prod_county_wide_df, dev_county_wide_df]).drop_duplicates(subset = cols, keep=False)

In [15]:
len(diff)

56

In [23]:
capacity_cols = [
    "fossil_existing_capacity_mw",
    "fossil_proposed_capacity_mw",
    "renewable_and_battery_existing_capacity_mw",
    "renewable_and_battery_proposed_capacity_mw",
    "battery_storage_existing_capacity_mw",
    "battery_storage_proposed_capacity_mw"
]
base_cols = ["county_id_fips", "county", "dev_prod"]

In [24]:
diff.sort_values(by="county_id_fips")[base_cols + capacity_cols]

,county_id_fips,county,dev_prod,fossil_existing_capacity_mw,fossil_proposed_capacity_mw,renewable_and_battery_existing_capacity_mw,renewable_and_battery_proposed_capacity_mw,battery_storage_existing_capacity_mw,battery_storage_proposed_capacity_mw
624,08031,Denver,prod,193.999996,0.00,20.300000,0.000,NaN,NaN
624,08031,Denver,dev,193.999996,0.00,24.800000,0.000,NaN,NaN
724,17037,DeKalb,prod,0.000000,0.00,4.000000,990.435,NaN,100.000
724,17037,DeKalb,dev,0.000000,0.00,4.000000,990.435,NaN,100.000
3059,23003,Aroostook,prod,0.000000,0.00,255.700000,2854.400,NaN,NaN
3059,23003,Aroostook,dev,0.000000,0.00,255.700000,2854.400,NaN,NaN
3127,23025,Somerset,prod,0.000000,0.00,94.500000,469.005,6.400000,NaN
3127,23025,Somerset,dev,0.000000,0.00,94.500000,469.005,6.400000,NaN
1032,25003,Berkshire,prod,182.500004,0.00,138.900000,905.978,NaN,869.970
1032,25003,Berkshire,dev,182.500004,0.00,138.900000,905.978,NaN,869.970


In [27]:
numeric_cols = dev_county_wide_df.select_dtypes(include=['number']).columns

In [41]:
changed_prod = prod_county_wide_df[prod_county_wide_df.county_id_fips.isin(diff.county_id_fips.unique())]
# we are going to be dividing by 0
changed_prod[numeric_cols] = changed_prod[numeric_cols].replace(0, np.nan)

changed_dev = dev_county_wide_df[dev_county_wide_df.county_id_fips.isin(diff.county_id_fips.unique())]

/tmp/ipykernel_83/1967672047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed_prod[numeric_cols] = changed_prod[numeric_cols].replace(0, np.nan)


In [42]:
merged = changed_prod.merge(changed_dev, on="county_id_fips", suffixes=("_prod", "_dev"))

In [46]:
prod_vals = merged[[f"{c}_prod" for c in numeric_cols]].values
dev_vals  = merged[[f"{c}_dev" for c in numeric_cols]].values

# Vectorized percent change
pct_change_vals = (dev_vals - prod_vals) / prod_vals * 100

# Wrap back into DataFrame
pct_change = pd.DataFrame(
    pct_change_vals,
    columns=[f"{c}_pct_change" for c in numeric_cols],
    index=merged.index
)
result = pd.concat([merged[["county_id_fips"]], pct_change], axis=1)

In [55]:
results_capacity_cols = [col for col in result.columns if "capacity_mw" in col]

In [56]:
result[["county_id_fips"] + results_capacity_cols]

,county_id_fips,fossil_existing_capacity_mw_pct_change,fossil_proposed_capacity_mw_pct_change,renewable_and_battery_existing_capacity_mw_pct_change,renewable_and_battery_proposed_capacity_mw_pct_change,renewable_and_battery_proposed_capacity_mw_actionable_pct_change,renewable_and_battery_proposed_capacity_mw_nearly_certain_pct_change,battery_storage_existing_capacity_mw_pct_change,battery_storage_proposed_capacity_mw_pct_change,coal_existing_capacity_mw_pct_change,coal_proposed_capacity_mw_pct_change,gas_existing_capacity_mw_pct_change,gas_proposed_capacity_mw_pct_change,offshore_wind_existing_capacity_mw_pct_change,offshore_wind_proposed_capacity_mw_pct_change,offshore_wind_capacity_mw_via_ports_pct_change,offshore_wind_proposed_capacity_mw_actionable_pct_change,offshore_wind_proposed_capacity_mw_nearly_certain_pct_change,oil_existing_capacity_mw_pct_change,oil_proposed_capacity_mw_pct_change,onshore_wind_existing_capacity_mw_pct_change,onshore_wind_proposed_capacity_mw_pct_change,onshore_wind_proposed_capacity_mw_actionable_pct_change,onshore_wind_proposed_capacity_mw_nearly_certain_pct_change,solar_existing_capacity_mw_pct_change,solar_proposed_capacity_mw_pct_change,solar_proposed_capacity_mw_actionable_pct_change,solar_proposed_capacity_mw_nearly_certain_pct_change
0,48027,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,48309,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,08031,0.0,NaN,22.167488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.167488,NaN,NaN,NaN
3,17037,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
4,48257,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
5,48131,NaN,NaN,NaN,-0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,-0.0,0.0,0.0
6,48159,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
7,25013,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,-0.0,NaN,NaN
8,51037,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
9,25003,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [57]:
capacity_cols = [col for col in dev_county_wide_df if "capacity_mw" in col]

In [60]:
(dev_county_wide_df[capacity_cols].sum() - prod_county_wide_df[capacity_cols].sum())/prod_county_wide_df[capacity_cols].sum()

fossil_existing_capacity_mw                                  0.000000
fossil_proposed_capacity_mw                                  0.000000
renewable_and_battery_existing_capacity_mw                   0.000015
renewable_and_battery_proposed_capacity_mw                   0.000000
renewable_and_battery_proposed_capacity_mw_actionable        0.000000
renewable_and_battery_proposed_capacity_mw_nearly_certain    0.000000
battery_storage_existing_capacity_mw                         0.000000
battery_storage_proposed_capacity_mw                         0.000000
coal_existing_capacity_mw                                    0.000000
coal_proposed_capacity_mw                                    0.000000
gas_existing_capacity_mw                                     0.000000
gas_proposed_capacity_mw                                     0.000000
offshore_wind_existing_capacity_mw                           0.000000
offshore_wind_proposed_capacity_mw                           0.000000
offshore_wind_capaci

In [62]:
num_cols_pct_chng = (dev_county_wide_df[numeric_cols].sum() - prod_county_wide_df[numeric_cols].sum())/prod_county_wide_df[numeric_cols].sum()

In [63]:
num_cols_pct_chng[num_cols_pct_chng != 0]

renewable_and_battery_existing_capacity_mw       0.000015
renewable_and_battery_existing_facility_count    0.000126
battery_storage_existing_co2e_tonnes_per_year         NaN
solar_existing_capacity_mw                       0.000034
solar_existing_facility_count                    0.000160
dtype: float64

In [ ]:
prod_county_long_sql = """
    SELECT *
    FROM `dbcp-dev-350818.data_mart.counties_wid_format`
"""

prod_county_wide_df = client.query_and_wait(prod_county_wide_sql).to_dataframe()